<font size=5>**Part I Background and Motivation**</font> <br/>

The body fat percentage (BFP) of a human is defined as the total mass of fat divided by total body mass, multiplied by 100. BFP is of great importance in measuring the well-being of individuals and predicting the risk of illness. However, the measurement of BFP has faced a trade-off problem between measurement accuracy and approach flexibility for a long time. Some methods that involve underwater measuring may not be generalized to people's daily life, while other methods which must consider a large number of variables neither is an idealized choice. 
Therefore finding a method that achieves both simplicity and exactitude in calculating BFP is our ultimate goal in conducting this project.

<font size=5>**Part II Data Introduction**</font> <br/>

The dataset obtained has a total of 252 observations with 17 features including their percentage of body fat cal and various body circumference measurements. The weight and height data are measured in a quarter unit, adiposity is obtained according to a specific formula mentioned in the following part, while other circumferences data are measured in 0.1 unit.

Since our goal for conducting this project is to find a “rule-of-thumb” approach achieved both simplicity, accuracy and robust, we consider the multiple linear regression as our basic model.

<font size=5>**Part III Data Preprocessing**</font> <br/>

In the data preprocessing part, we first examine whether the data has abnormal observations from the following 4 aspects:
1. According to the histogram plot of `BODYFAT`, we find that observation `IDNO=182` has feature `BODYFAT=0`, which is apparently can not happen in the real world, therefore we remove this observation.
2. As Siri's equation shown, `BODYFAT` goes linear with `1/DENSITY`. After fitting the linear regression model between `BODYFAT` and `1/DENSITY`  we find observations with `IDNO=48,76,96` are outliers according to remedies plots. Since we can not decide neither `BODYFAT` nor `1/DENSITY` is trustworthy, we choose to delete these three points.
3. According to the calculation formula of BMI (BMI=703Weight/Height^2), we find observation `IDNO=42` has the wrong height, we remove this observation and re-fit the linear model between `ADIPOSITY` and `WEIGHT/HEIGHT^2`. According to the remedies plots of the re-fitted model, observations with `IDNO=163,221` are outliers. After looking through the whole data, we cannot determine which feature's value is convincing and finally remove these two outliers. As for observation `IDNO=42`, we use the re-fitted model to correct its height.
4. Except for the above approaches, we also take a look at other features. The correlation plot shows that there is strong multicollinearity between certain variables. The observation with `IDNO=39` has extremely large values in certain features. While we think this kind of person may truly exist, we choose to keep this observation in our dataset.

<table>
 <td> 
<img src="../figure/fitted_density_model.jpeg" width="90%"> <br>
</td> 
<td> 
<img src="../figure/fitted_raw_wh_model.jpeg" width="90%"> <br>
</td> 
<td> 
<img src="../figure/refitted_wh_model.jpeg" width="90%"> <br>
</td> 
</table>

After removing several suspectable observations, we fit the full model with remain data and do further detection for the linear regression model. It can be shown from the remedies plots of the full model that there are no obvious outliers anymore. Except for outliers, we make a normal Q-Q plot for residuals and also perform Box-Cox transformation of the full model. Both of these two methods show that remained data are good enough so we do not need to do the transformation.

<font size=5>**Part IV Variable Selection**</font> <br/>

<font size=4>**4.1 Mallow's Cp**</font> <br/>
Below are the results,model which choose different variables will get different Adjusted R-squared values: <br/>

|model|Adjusted $R^2$|
|:--|:-------------:|
|BODYFAT ~ ABDOMEN |0.659|
|BODYFAT ~ ABDOMEN+WEIGHT | 0.718 |
|BODYFAT ~ ABDOMEN+WEIGHT+WRIST | 0.724|
|BODYFAT ~ ABDOMEN+WEIGHT+WRIST+FOREARM | 0.731|

<font size=4>**4.2 AIC & BIC**</font> <br/>



|Criteria | Model | Adjusted $R^2$|
|:-------:|:-------|:---:|
|| **Both direction**| |
|AIC | BODYFAT ~ AGE + WEIGHT + HEIGHT + ADIPOSITY + NECK +ABDOMEN + HIP + THIGH +FOREARM + WRIST | 0.7413|
|BIC | BODYFAT ~ WEIGHT + ABDOMEN + FOREARM + WRIST | 0.7318|
| |**Backward direction** | |
|AIC | BODYFAT ~ AGE + WEIGHT + HEIGHT + ADIPOSITY + NECK +ABDOMEN + HIP + THIGH + FOREARM + WRIST | 0.7413 |
|BIC | BODYFAT ~ WEIGHT + ABDOMEN + FOREARM + WRIST | 0.7318 |
| |**Forward direction** | |
|AIC | BODYFAT ~ ABDOMEN + WEIGHT + WRIST + FOREARM + NECK + BICEPS | 0.7349 |
|BIC | BODYFAT ~ WEIGHT + ABDOMEN + FOREARM + WRIST |0.7318|

<font size=4>**4.3 F-test**</font> <br/>
Factors in final model using this method: <br/>
BODYFAT ~ WEIGHT+HEIGHT+ADIPOSITY+ABDOMEN+THIGH+FOREARM+WRIST    <br/> 

<font size=4>**4.4 LASSO**</font> <br/>
In order to deal with the problem of multicolinearity and do variable feature selection to choose a simpler model, we used lasso with lambda computed according to 1se rule. Then, we get 6 variables for the model:<br/>
BODYFAT ~ AGE+HEIGHT+NECK+ABDOMEN+FOREARM+WRIST<br/>
Even though the multicolinearity has been reduced, the model is not simple enough. Therefore, we do not choose LASSO as our final model.<br/>

<font size=4>**4.5 Cross-Validation Based on Selected Variables**</font> <br/>
According to the result based on above citeria, we find that WEIGHT,ABDOMEN,WRIST,FOREARM are most selected. In order to further detect necessary variables in our final model, we use both $R^2$ and Cross-Validation to evaluate different model constructed by these variables.<br/>

Following are the results:

|No.|Variable Numbers |            Model                 | 10-fold-CV | $R^2$ |
|:-:|:---------------:|:---------------------------------|:----------:|:--------------:|
| 1 |       2         | BODYFAT ~ ABDOMEN+WEIGHT         |  17.8358  |   0.7203    |
| 2 |       2         | BODYFAT ~ ABDOMEN+WRIST          |  17.8831   |  0.6990     |
| 3 |       2         | BODYFAT ~ ABDOMEN+FOREARM        |  20.7023  |   0.6631    |
| 4 |       2         | BODYFAT ~ WEIGHT+WRIST           |  42.3430  |  0.3894     |
| 5 |       2         | BODYFAT ~ WEIGHT+FOREARM         |  45.9751  |   0.3713    |
| 6 |       2         | BODYFAT ~ WRIST+FOREARM          |  64.6050  |  0.1588     |
| 7 |       3         | BODYFAT ~ ABDOMEN+WEIGHT+WRIST   |  17.5758  |   0.7276    |
| 8 |       3         | BODYFAT ~ ABDOMEN+WEIGHT+FOREARM |  17.2560  |   0.7255    |
| 9 |       3         | BODYFAT ~ WEIGHT+WRIST+FOREARM   |  42.3504  |  0.3894     |
| 10|       4         | BODYFAT ~ ABDOMEN+WEIGHT+WRIST+FOREARM|  16.8968  |    0.7362   |

<font size=5>**Part V Final Model & Outliers**</font> <br/>

<font size=4>**3.1 Final model description**</font> <br/>
We choose weight and abdomen as predictiors for body fat as our final model.

In [8]:
lm.reg=lm(BODYFAT~WEIGHT+ABDOMEN,fat.rm)

Our final model is 
$$\text{BodyFat} = -42.95790 -0.11994 \cdot \text{WEIGHT} + 0.90152 \cdot \text{ABDOMEN} $$
The p-values of the coefficients are all much smaller than 0.01. Therefore, coefficients are significant.The adjusted R-squared is 0.7148. The residual standard error is 4.083.<br/>
A laymen’s interpretation is that when your weight increases one additinal unit(lbs), the bodyfat will decrease 0.12%. When abdomen increases one additional unit(cm), the bodyfat will increase 0.90%.

<font size=4>**3.2 Advantages and disadvantages**</font> <br/>
**Advantages**: The model is very simple since there are only two variables and it is convenient for users to gain their weight and abdomen data. The multicolinearity has been reduced and the model is robust. <br/>
**Disadvantages**: Since there are only two variables, we sacrificed some accuracy of this model. The model is based on the data of male ones within age 22-81, which means it may not predict females out of the age range well. Besides, there are still some outliers. Meanwhile, the data of users' abdomen may not be as convenient as variables like height and weight to obtain.

<font size=4>**3.3 Model Diagnose & Outliers**</font> <br/>
<font size=3>**3.3.1 Assumption diagnose and outlier detection**</font> <br/>
To futherly estimate our model, we draw plots to check the assumptions and outliers for this model. As we can see from the residuals vs fitted values plot, the assumpiton of equal variance and linearity have been satisfied. Besides, From the qqplot, we can see that the normality assumption has almostly been satisfied.We also drew standard / studentized residuals vs fitted values plots and find that there is no significant outlier.

<table>
 <td> 
<img src="../figure/res.png" width="70%"> <br>
</td> 
<td> 
<img src="../figure/qq.png" width="50%"> <br>
</td> 
<td> 
<img src="../figure/Leverage.png" width="140%"> <br>
</td> 
<td> 
<img src="../figure/DFFITS.png" width="100%"> <br>
</td> 
</table>

<font size=3>**3.3.2 Influential points detection** </font> <br/>
To determine some influential points, we draw Leverage plot, Diffits plot, Cook's distance plot and Debates plot. Aside from Leverage plot, the other three plots does not find influential points. From the Leverage plot, we find 8 influential points(9,12,35,36,152,216,242th). However, there is no enough evidence showing that they are outliers even though they are infulential.


<font size=5>**Part VI Conclusion**</font> <br/>

Our conclusion is that we can predict someone body fat according to his/her weight and abdomen. 
$$\text{BodyFat} = -42.95790 -0.11994 \cdot \text{WEIGHT} + 0.90152 \cdot \text{ABDOMEN} $$
The heavier one is, the smaller his/her body fat percentage is. The larger his/her abdomen value is, the larger his/her body fat percentage is.